In [1]:
import pandas as pd

In [2]:
df_test = pd.read_csv('test-annotations-object-segmentation.csv')

In [3]:
df_test.head()

,MaskPath,ImageID,LabelName,BoxID,BoxXMin,BoxXMax,BoxYMin,BoxYMax,PredictedIoU,Clicks
0,d0ed76e0533a914d_m01xyhv_cffd8afa.png,d0ed76e0533a914d,/m/01xyhv,cffd8afa,0.122966,0.958409,0.389892,0.998195,0.0,NaN
1,fba940ee0203b368_m08pbxl_13094a08.png,fba940ee0203b368,/m/08pbxl,13094a08,0.460938,0.551562,0.387500,0.572917,0.0,NaN
2,10946fa016098d27_m0k4j_ec8644a0.png,10946fa016098d27,/m/0k4j,ec8644a0,0.203125,0.309375,0.402083,0.514583,0.0,NaN
3,fcb0b452b40f3529_m01n4qj_f1fedfea.png,fcb0b452b40f3529,/m/01n4qj,f1fedfea,0.238806,0.650320,0.330275,0.727829,0.0,NaN
4,34d9ee0ec37f5078_m021mn_54e3a6b6.png,34d9ee0ec37f5078,/m/021mn,54e3a6b6,0.135693,0.382006,0.139381,0.314159,0.0,NaN


In [4]:
df_train = pd.read_csv('train-annotations-object-segmentation.csv')

In [7]:
df_train["LabelName"].value_counts()[:10]

LabelName
/m/09j2d      414489
/m/05r655     173787
/m/0k4j       159435
/m/083wq      155416
/m/01g317     149166
/m/02p0tk3    135182
/m/03bt1vf    117234
/m/04yx4      113747
/m/01xyhv     100410
/m/0c9ph5      89784
Name: count, dtype: int64

In [9]:
df_class = pd.read_csv('oidv7-class-descriptions.csv')

In [10]:
df_class.head()

,LabelName,DisplayName
0,/m/0c4936,'Nduja
1,/m/06w6y06,10 cane
2,/m/079zcf,100 metres hurdles
3,/m/02pv3hz,100plus
4,/m/0bb154,110 metres hurdles


In [18]:
classes = df_train["LabelName"].value_counts()[:25].index.tolist()
classes_names = df_class[df_class["LabelName"].isin(classes)]

In [19]:
classes_names

,LabelName,DisplayName
1883,/m/01bqk0,Bicycle wheel
1940,/m/015p6,Bird
2447,/m/04dr76w,Bottle
2519,/m/01bl7v,Boy
3303,/m/0k4j,Car
4322,/m/09j2d,Clothing
5775,/m/0bt9lr,Dog
5959,/m/01d40f,Dress
5987,/m/0271t,Drink
7058,/m/0cmf2,Fixed-wing aircraft


In [23]:
desired_class_names = ["Bird", "Bottle", "Sculpture", "Flower"]
desidered_class_ids = df_class[df_class["DisplayName"].isin(desired_class_names)]["LabelName"].tolist()
desidered_class_ids

['/m/015p6', '/m/04dr76w', '/m/0c9ph5', '/m/06msq']

In [33]:
desired_images_ids = df_train[df_train["LabelName"].isin(desidered_class_ids)]["ImageID"].unique()
desired_images_ids.shape, desired_images_ids[:5]

((70316,),
 array(['339e1aea3fd20101', 'e71037b27a6d8bb5', 'a2c3448cdecc0877',
        '6f066dee08fd1e16', '09991c4d538fb114'], dtype=object))

In [37]:
other_desired_images_ids = df_train[~df_train["LabelName"].isin(desidered_class_ids)].sample(200000)["ImageID"].unique()[:70316]
other_desired_images_ids.shape, other_desired_images_ids[:5]

((70316,),
 array(['aa8fef2e2fcd61b6', 'd28451ecc0ab56e7', '8d73d2f251f29794',
        '829d3dd4e6d3291c', '3f97685a2c9b967a'], dtype=object))

In [38]:
all_images_ids = list(desired_images_ids) + list(other_desired_images_ids)
len(all_images_ids)
with open('selected_image_ids.txt', 'w') as f:
    for item in all_images_ids:
        f.write("%s\n" % item)

In [39]:
df_test = pd.read_csv('test-annotations-object-segmentation.csv')
df_val = pd.read_csv('validation-annotations-object-segmentation.csv')

test_image_ids = df_test[df_test["LabelName"].isin(desidered_class_ids)]["ImageID"].unique()
val_image_ids = df_val[df_val["LabelName"].isin(desidered_class_ids)]["ImageID"].unique()

test_image_ids.shape, val_image_ids.shape

((2304,), (765,))

In [40]:
test_image_ids[:5], val_image_ids[:5], desidered_class_ids

(array(['58ae9ada2cec71b1', 'b08327c40a8b02bb', 'caa44bce1d51cee3',
        '1d376c215f26fffa', '8ca2aaaf1c23b66c'], dtype=object),
 array(['a0f9c872b965227a', '4f5377a910cfff8f', 'a119ed96a3eb26da',
        'e615472ab4c2d73e', '408b918f0d724046'], dtype=object),
 ['/m/015p6', '/m/04dr76w', '/m/0c9ph5', '/m/06msq'])

In [41]:
with open('test_selected_image_ids.txt', 'w') as f:
    for item in test_image_ids:
        f.write("%s\n" % item)

with open('val_selected_image_ids.txt', 'w') as f:
    for item in val_image_ids:
        f.write("%s\n" % item)

In [45]:
class0_annnotated_image_ids = df_train[df_train["LabelName"] == desidered_class_ids[0]]["ImageID"].sample(500).to_list()
class1_annnotated_image_ids = df_train[df_train["LabelName"] == desidered_class_ids[1]]["ImageID"].sample(500).to_list()
class2_annnotated_image_ids = df_train[df_train["LabelName"] == desidered_class_ids[2]]["ImageID"].sample(500).to_list()
class3_annnotated_image_ids = df_train[df_train["LabelName"] == desidered_class_ids[3]]["ImageID"].sample(500).to_list()

len(class0_annnotated_image_ids), len(class1_annnotated_image_ids), len(class2_annnotated_image_ids), len(class3_annnotated_image_ids)

(500, 500, 500, 500)

In [47]:
downtream_train_ids = class0_annnotated_image_ids + class1_annnotated_image_ids + class2_annnotated_image_ids + class3_annnotated_image_ids
len(downtream_train_ids)

2000

In [48]:
with open('downstream_train_image_ids.txt', 'w') as f:
    for item in downtream_train_ids:
        f.write("%s\n" % item)

In [51]:
df_train[df_train["ImageID"].isin(downtream_train_ids)][:5]

,MaskPath,ImageID,LabelName,BoxID,BoxXMin,BoxXMax,BoxYMin,BoxYMax,PredictedIoU,Clicks
48,e71037b27a6d8bb5_m0c9ph5_29dbe880.png,e71037b27a6d8bb5,/m/0c9ph5,29dbe880,0.476250,0.663125,0.585000,0.862500,0.91094,0.59952 0.61219 0;0.49186 0.72990 0;0.51196 0....
245,5fc7dd2ca1ec2667_m04dr76w_7a5b2700.png,5fc7dd2ca1ec2667,/m/04dr76w,7a5b2700,0.546250,0.680625,0.335000,0.785833,0.77915,0.66833 0.74226 1;0.68401 0.48899 0;0.60362 0....
266,b1270151a9721787_m015p6_bb501f55.png,b1270151a9721787,/m/015p6,bb501f55,0.170898,0.292969,0.582031,0.640625,0.67821,0.19669 0.59138 0;0.26754 0.62846 0;0.24407 0....
456,fcb8ab72759156f8_m04dr76w_aafccbb2.png,fcb8ab72759156f8,/m/04dr76w,aafccbb2,0.610000,0.675000,0.390000,0.463333,0.67588,0.65637 0.41791 0;0.65632 0.41609 1;0.66594 0....
507,3ce703aefec000b9_m04dr76w_34a5494b.png,3ce703aefec000b9,/m/04dr76w,34a5494b,0.376250,0.404375,0.109244,0.192437,0.76243,0.38059 0.14577 0;0.39020 0.15870 0;0.38436 0....


In [56]:
df_class[df_class["LabelName"].isin(desidered_class_ids)]["DisplayName"].unique()

array(['Bird', 'Bottle', 'Flower', 'Sculpture'], dtype=object)

In [60]:
df_train[df_train["ImageID"].isin(downtream_train_ids[0:1])][df_train["LabelName"].isin(desidered_class_ids)]

/tmp/ipykernel_10550/69173819.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_train[df_train["ImageID"].isin(downtream_train_ids[0:1])][df_train["LabelName"].isin(desidered_class_ids)]


,MaskPath,ImageID,LabelName,BoxID,BoxXMin,BoxXMax,BoxYMin,BoxYMax,PredictedIoU,Clicks
213925,6230c0bc75d4da60_m015p6_139ba147.png,6230c0bc75d4da60,/m/015p6,139ba147,0.841250,0.916250,0.737500,0.784167,0.84882,0.87878 0.78415 0;0.84726 0.73943 0;0.88193 0....
271942,6230c0bc75d4da60_m015p6_cb29d4c2.png,6230c0bc75d4da60,/m/015p6,cb29d4c2,0.600000,0.650000,0.610833,0.648333,0.81004,0.63818 0.63172 0;0.59995 0.65053 0;0.61234 0....
320427,6230c0bc75d4da60_m015p6_3de11a01.png,6230c0bc75d4da60,/m/015p6,3de11a01,0.704375,0.766250,0.748333,0.791667,0.77009,0.72111 0.78295 0;0.74417 0.78210 1;0.70937 0....
355165,6230c0bc75d4da60_m015p6_b53da8a0.png,6230c0bc75d4da60,/m/015p6,b53da8a0,0.936875,0.993750,0.625833,0.690000,0.71452,0.93731 0.63892 1;0.98470 0.65842 0;0.94064 0....
396904,6230c0bc75d4da60_m015p6_9aa857c8.png,6230c0bc75d4da60,/m/015p6,9aa857c8,0.395000,0.473750,0.734167,0.860833,0.78537,0.45999 0.76689 0;0.43557 0.84899 1;0.39296 0....
558111,6230c0bc75d4da60_m015p6_dbeeb700.png,6230c0bc75d4da60,/m/015p6,dbeeb700,0.186250,0.236875,0.483333,0.565833,0.87298,0.18624 0.57140 1;0.18294 0.55336 1;0.22290 0....
697612,6230c0bc75d4da60_m015p6_7017a95d.png,6230c0bc75d4da60,/m/015p6,7017a95d,0.285625,0.350625,0.698333,0.741667,0.87128,0.29087 0.71251 1;0.32410 0.74083 1;0.35187 0....
1097354,6230c0bc75d4da60_m015p6_5f4d7587.png,6230c0bc75d4da60,/m/015p6,5f4d7587,0.551875,0.633125,0.760000,0.853333,0.81760,0.55690 0.77351 0;0.59240 0.83412 0;0.58468 0....
1127213,6230c0bc75d4da60_m015p6_d962b7e2.png,6230c0bc75d4da60,/m/015p6,d962b7e2,0.491250,0.543125,0.674167,0.722500,0.90394,0.48832 0.70944 0;0.53378 0.68556 0;0.50662 0....
1138808,6230c0bc75d4da60_m015p6_e80ef53d.png,6230c0bc75d4da60,/m/015p6,e80ef53d,0.169375,0.248125,0.724167,0.785833,0.82269,0.17823 0.73865 1;0.19860 0.77164 1;0.24372 0....


In [61]:
def prefixar_linhas_em_arquivo(nome_arquivo, prefixo):
    """
    Lê um arquivo de texto linha por linha, adiciona um prefixo a cada linha
    (removendo espaços em branco ou quebras de linha existentes) e
    sobrescreve o arquivo com as novas linhas.

    :param nome_arquivo: O caminho para o arquivo de texto.
    :param prefixo: A string a ser adicionada no início de cada linha.
    """

    # 1. Ler o conteúdo do arquivo
    try:
        with open(nome_arquivo, 'r') as f:
            linhas = f.readlines()
    except FileNotFoundError:
        print(f"Erro: O arquivo '{nome_arquivo}' não foi encontrado.")
        return
    except Exception as e:
        print(f"Ocorreu um erro ao ler o arquivo: {e}")
        return

    # 2. Processar as linhas (adicionar o prefixo)
    novo_conteudo = []
    # O rstrip() remove quebras de linha e espaços em branco à direita
    # antes de adicionar o prefixo e a nova quebra de linha.
    for linha in linhas:
        linha_limpa = linha.rstrip()
        # Evita adicionar o prefixo em linhas completamente vazias
        if linha_limpa:
            nova_linha = f"{prefixo}/{linha_limpa}\n"
        else:
            nova_linha = "\n"  # Mantém linhas vazias como estão
        novo_conteudo.append(nova_linha)


    # 3. Sobrescrever o arquivo com o novo conteúdo
    try:
        # Abre o arquivo no modo 'w' (escrita), o que sobrescreve o conteúdo existente
        with open(nome_arquivo, 'w') as f:
            f.writelines(novo_conteudo)
        print(f"O arquivo '{nome_arquivo}' foi atualizado com sucesso.")

    except Exception as e:
        print(f"Ocorreu um erro ao escrever no arquivo: {e}")

In [66]:
prefixar_linhas_em_arquivo("dataset/test-ids.txt", "test")

O arquivo 'dataset/test-ids.txt' foi atualizado com sucesso.
